<a href="https://colab.research.google.com/github/jamessutton600613-png/GC/blob/main/Untitled287.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import time

# ============================================================
# SWEEP CONFIG (RANGES)
# ============================================================

RANGES = {
    # Core scale
    "N_PATHS_LIST": np.array([10, 20, 50, 100, 200, 500, 1000], dtype=int),

    # Thermodynamic sensitivity
    "BETA_LIST": np.array([0.8, 1.2, 1.8, 2.2, 3.0, 4.0], dtype=float),

    # Disorder
    "DISORDER_JUMP_LIST": np.array([0.0, 0.1, 0.2, 0.3, 0.5, 0.7], dtype=float),
    "SHOCK_MODE_LIST": np.array([0, 1, 2], dtype=int),  # 0: none, 1: one shock @25%, 2: shocks @25% and @60%

    # GQR controls
    "GQR_FILTER_BASE_LIST": np.array([0.4, 0.6, 0.75, 0.9, 0.97], dtype=float),
    "GQR_FILTER_COST_LIST": np.array([1e-4, 3e-4, 8e-4, 1.6e-3, 3.2e-3, 6.4e-3], dtype=float),
    "GQR_EARN_SCALE_LIST": np.array([0.0, 0.01, 0.03, 0.05, 0.08, 0.12], dtype=float),
    "GQR_BUDGET0_LIST": np.array([0.5, 1.0, 2.0, 4.0], dtype=float),
    "GQR_BUDGET_MIN_LIST": np.array([-0.5, -2.0, -6.0], dtype=float),
    "GQR_BUDGET_MAX_LIST": np.array([2.0, 4.0, 8.0], dtype=float),
    "JURIS_THRESHOLD_LIST": np.array([0.2, 0.35, 0.45, 0.6], dtype=float),

    # Landscape distribution parameters (broad, generic)
    "BARRIER_MU_LIST": np.array([0.8, 1.0, 1.2, 1.5], dtype=float),
    "BARRIER_SIGMA_LIST": np.array([0.10, 0.25, 0.40], dtype=float),
    "REORG_MU_LIST": np.array([0.15, 0.35, 0.55], dtype=float),
    "REORG_SIGMA_LIST": np.array([0.05, 0.10, 0.18], dtype=float),
    "DISS_MU_LIST": np.array([0.6, 1.0, 1.4], dtype=float),
    "DISS_SIGMA_LIST": np.array([0.15, 0.25, 0.45], dtype=float),

    # Product specialness offsets (relative to sampled distributions)
    "PROD_BARRIER_OFFSET_LIST": np.array([-0.5, -0.25, 0.0, 0.25], dtype=float),
    "PROD_DISS_OFFSET_LIST": np.array([-0.5, -0.25, 0.0, 0.25], dtype=float),

    # Warshel knobs (optional; included for robustness comparisons)
    "WARSHEL_LOWER_PRODUCT_LIST": np.array([0.0, 0.1, 0.25, 0.4], dtype=float),
    "WARSHEL_RAISE_WASTE_LIST": np.array([0.0, 0.03, 0.05, 0.1], dtype=float),
    "WARSHEL_DISS_REDUCE_LIST": np.array([0.0, 0.03, 0.05, 0.1], dtype=float),
}

# ============================================================
# RUN SETTINGS (edit these)
# ============================================================

OUTFILE = "enz_governance_sweep_npz_only.npz"

N_SAMPLES = 400                 # number of parameter sets to sample (increase later)
LANDSCAPES_PER_SAMPLE = 12       # replicate landscapes per parameter set
SEEDS_PER_LANDSCAPE = 2          # replicate seeds per landscape
SEED0 = 123

# Chunked simulation settings (fast)
EVENTS = 80_000
CHUNK = 1_000
N_CHUNKS = EVENTS // CHUNK
PRODUCT_IDX = 0

# Budget bookkeeping overhead constants (these match the style in your earlier code)
GQR_BASE_OVERHEAD = 0.0003       # per event
FILTER_DEAD_THRESH = 0.05        # effective filter below this => gqr_dead=1

# For numerical stability / clipping
BARRIER_CLIP = (0.05, 4.0)
REORG_CLIP = (0.0, 2.0)
DISS_CLIP = (0.0, 4.0)

# ============================================================
# Utility: sampling
# ============================================================

rng_master = np.random.default_rng(SEED0)

def pick(rng, arr):
    return arr[int(rng.integers(0, len(arr)))]

def sample_param_set(rng):
    return {
        "N_PATHS": int(pick(rng, RANGES["N_PATHS_LIST"])),
        "BETA": float(pick(rng, RANGES["BETA_LIST"])),
        "DISORDER_JUMP": float(pick(rng, RANGES["DISORDER_JUMP_LIST"])),
        "SHOCK_MODE": int(pick(rng, RANGES["SHOCK_MODE_LIST"])),

        "GQR_FILTER_BASE": float(pick(rng, RANGES["GQR_FILTER_BASE_LIST"])),
        "GQR_FILTER_COST": float(pick(rng, RANGES["GQR_FILTER_COST_LIST"])),
        "GQR_EARN_SCALE": float(pick(rng, RANGES["GQR_EARN_SCALE_LIST"])),
        "GQR_BUDGET0": float(pick(rng, RANGES["GQR_BUDGET0_LIST"])),
        "GQR_BUDGET_MIN": float(pick(rng, RANGES["GQR_BUDGET_MIN_LIST"])),
        "GQR_BUDGET_MAX": float(pick(rng, RANGES["GQR_BUDGET_MAX_LIST"])),
        "JURIS_THRESHOLD": float(pick(rng, RANGES["JURIS_THRESHOLD_LIST"])),

        "BARRIER_MU": float(pick(rng, RANGES["BARRIER_MU_LIST"])),
        "BARRIER_SIGMA": float(pick(rng, RANGES["BARRIER_SIGMA_LIST"])),
        "REORG_MU": float(pick(rng, RANGES["REORG_MU_LIST"])),
        "REORG_SIGMA": float(pick(rng, RANGES["REORG_SIGMA_LIST"])),
        "DISS_MU": float(pick(rng, RANGES["DISS_MU_LIST"])),
        "DISS_SIGMA": float(pick(rng, RANGES["DISS_SIGMA_LIST"])),

        "PROD_BARRIER_OFFSET": float(pick(rng, RANGES["PROD_BARRIER_OFFSET_LIST"])),
        "PROD_DISS_OFFSET": float(pick(rng, RANGES["PROD_DISS_OFFSET_LIST"])),

        "WARSHEL_LOWER_PRODUCT": float(pick(rng, RANGES["WARSHEL_LOWER_PRODUCT_LIST"])),
        "WARSHEL_RAISE_WASTE": float(pick(rng, RANGES["WARSHEL_RAISE_WASTE_LIST"])),
        "WARSHEL_DISS_REDUCE": float(pick(rng, RANGES["WARSHEL_DISS_REDUCE_LIST"])),
    }

# ============================================================
# Landscape draw
# ============================================================

def draw_landscape(rng, N_PATHS, mu_b, sig_b, mu_r, sig_r, mu_d, sig_d, prod_bar_off, prod_diss_off):
    barriers = np.clip(rng.normal(mu_b, sig_b, size=N_PATHS), *BARRIER_CLIP)
    reorg = np.clip(rng.normal(mu_r, sig_r, size=N_PATHS), *REORG_CLIP)
    diss = np.clip(rng.normal(mu_d, sig_d, size=N_PATHS), *DISS_CLIP)

    # make product path slightly special relative to sampled distribution
    barriers[PRODUCT_IDX] = np.clip(barriers[PRODUCT_IDX] + prod_bar_off, *BARRIER_CLIP)
    reorg[PRODUCT_IDX] = np.clip(reorg[PRODUCT_IDX], *REORG_CLIP)
    diss[PRODUCT_IDX] = np.clip(diss[PRODUCT_IDX] + prod_diss_off, *DISS_CLIP)

    return barriers, reorg, diss

# ============================================================
# Weights
# ============================================================

def softmax_stable(x):
    x = x - np.max(x)
    w = np.exp(x)
    s = w.sum()
    return w / s if s > 0 else np.ones_like(w) / len(w)

def weights_arrhenius(barriers, BETA):
    return softmax_stable(-BETA * barriers)

def weights_marcus(barriers, reorg, BETA):
    return softmax_stable(-BETA * (barriers + reorg))

def weights_warshel(barriers, BETA, lower_product, raise_waste):
    b = barriers.copy()
    b[PRODUCT_IDX] = max(0.0, b[PRODUCT_IDX] - lower_product)
    if b.size > 1:
        b[1:] = b[1:] + raise_waste
    return softmax_stable(-BETA * b)

def gqr_effective_filter(filter_setting, disorder, budget, BUDGET_MIN):
    eff = filter_setting * max(0.0, 1.0 - disorder)
    if budget < 0:
        frac = max(0.0, (budget - BUDGET_MIN) / (0.0 - BUDGET_MIN))
        eff *= frac
    return float(np.clip(eff, 0.0, 1.0))

def weights_gqr(barriers, BETA, filter_eff):
    w = np.exp(-BETA * barriers)
    if w.size > 1:
        w[1:] *= (1.0 - filter_eff)
    s = w.sum()
    return (w / s) if s > 0 else np.ones_like(w) / len(w)

# ============================================================
# One run (chunked) for 4 regimes: Arr, Marcus, Warshel, GQR
# ============================================================

def run_once_chunked(params, barriers, reorg, diss, seed):
    rng = np.random.default_rng(seed)

    N_PATHS = int(params["N_PATHS"])
    BETA = float(params["BETA"])
    dj = float(params["DISORDER_JUMP"])
    shock_mode = int(params["SHOCK_MODE"])

    # shock schedule in chunks
    shock1 = int(0.25 * N_CHUNKS)
    shock2 = int(0.60 * N_CHUNKS)

    w_arr = weights_arrhenius(barriers, BETA)
    w_mar = weights_marcus(barriers, reorg, BETA)

    w_war = weights_warshel(
        barriers,
        BETA,
        float(params["WARSHEL_LOWER_PRODUCT"]),
        float(params["WARSHEL_RAISE_WASTE"]),
    )

    diss_warshel = np.maximum(0.0, diss - float(params["WARSHEL_DISS_REDUCE"]))

    # GQR
    budget = float(params["GQR_BUDGET0"])
    BUDGET_MIN = float(params["GQR_BUDGET_MIN"])
    BUDGET_MAX = float(params["GQR_BUDGET_MAX"])
    filter_base = float(params["GQR_FILTER_BASE"])
    filter_cost = float(params["GQR_FILTER_COST"])
    earn_scale = float(params["GQR_EARN_SCALE"])
    juris_thr = float(params["JURIS_THRESHOLD"])

    # tallies
    prod = np.zeros(4, dtype=float)
    waste = np.zeros(4, dtype=float)
    diss_tot = np.zeros(4, dtype=float)

    disorder = 0.0
    d_prod = float(diss[PRODUCT_IDX])

    for c in range(N_CHUNKS):
        if shock_mode == 1 and c == shock1:
            disorder = float(np.clip(disorder + dj, 0.0, 0.95))
        elif shock_mode == 2:
            if c == shock1 or c == shock2:
                disorder = float(np.clip(disorder + dj, 0.0, 0.95))

        # Arrhenius
        counts = rng.multinomial(CHUNK, w_arr)
        prod[0] += counts[PRODUCT_IDX]
        waste[0] += (CHUNK - counts[PRODUCT_IDX])
        diss_tot[0] += float((counts * diss).sum())

        # Marcus
        counts = rng.multinomial(CHUNK, w_mar)
        prod[1] += counts[PRODUCT_IDX]
        waste[1] += (CHUNK - counts[PRODUCT_IDX])
        diss_tot[1] += float((counts * diss).sum())

        # Warshel
        counts = rng.multinomial(CHUNK, w_war)
        prod[2] += counts[PRODUCT_IDX]
        waste[2] += (CHUNK - counts[PRODUCT_IDX])
        diss_tot[2] += float((counts * diss_warshel).sum())

        # GQR (dynamic)
        filter_setting = filter_base if disorder <= juris_thr else 0.65 * filter_base
        filter_eff = gqr_effective_filter(filter_setting, disorder, budget, BUDGET_MIN)
        w_gqr = weights_gqr(barriers, BETA, filter_eff)

        counts = rng.multinomial(CHUNK, w_gqr)
        got_prod = float(counts[PRODUCT_IDX])
        prod[3] += got_prod
        waste[3] += (CHUNK - counts[PRODUCT_IDX])
        diss_chunk = float((counts * diss).sum())
        diss_tot[3] += diss_chunk

        # Budget update (coarse but consistent with your earlier code style)
        spend = CHUNK * ((filter_cost * filter_eff) + GQR_BASE_OVERHEAD)
        earn = earn_scale * got_prod * max(0.0, 1.2 - d_prod)
        budget = float(np.clip(budget + earn - spend, BUDGET_MIN, BUDGET_MAX))

    y = prod / np.maximum(1.0, prod + waste)
    dpp = diss_tot / np.maximum(1.0, prod)

    final_filter = gqr_effective_filter(filter_base, disorder, budget, BUDGET_MIN)
    gqr_dead = 1 if final_filter < FILTER_DEAD_THRESH else 0

    return {
        "y_arr": float(y[0]), "y_mar": float(y[1]), "y_war": float(y[2]), "y_gqr": float(y[3]),
        "dpp_arr": float(dpp[0]), "dpp_mar": float(dpp[1]), "dpp_war": float(dpp[2]), "dpp_gqr": float(dpp[3]),
        "gqr_budget": float(budget),
        "gqr_final_filter": float(final_filter),
        "gqr_dead": int(gqr_dead),
        "final_disorder": float(disorder),
    }

# ============================================================
# SWEEP RUNNER (NPZ ONLY)
# ============================================================

samples = [sample_param_set(rng_master) for _ in range(N_SAMPLES)]

columns = [
    # sample_id + replication ids
    "sample_id", "landscape_id", "seed_id",

    # key params
    "N_PATHS", "BETA", "DISORDER_JUMP", "SHOCK_MODE",
    "GQR_FILTER_BASE", "GQR_FILTER_COST", "GQR_EARN_SCALE",
    "GQR_BUDGET0", "GQR_BUDGET_MIN", "GQR_BUDGET_MAX", "JURIS_THRESHOLD",
    "BARRIER_MU", "BARRIER_SIGMA", "REORG_MU", "REORG_SIGMA", "DISS_MU", "DISS_SIGMA",
    "PROD_BARRIER_OFFSET", "PROD_DISS_OFFSET",
    "WARSHEL_LOWER_PRODUCT", "WARSHEL_RAISE_WASTE", "WARSHEL_DISS_REDUCE",

    # outputs
    "y_arr", "y_mar", "y_war", "y_gqr",
    "dpp_arr", "dpp_mar", "dpp_war", "dpp_gqr",
    "gqr_budget", "gqr_final_filter", "gqr_dead", "final_disorder",
]

rows = []
t0 = time.time()
last_beat = t0

total_runs = N_SAMPLES * LANDSCAPES_PER_SAMPLE * SEEDS_PER_LANDSCAPE
run_count = 0

for si, p in enumerate(samples):
    for li in range(LANDSCAPES_PER_SAMPLE):
        # draw a new landscape per replicate
        seed_land = int(rng_master.integers(1, 2**31 - 1))
        rng_land = np.random.default_rng(seed_land)

        barriers, reorg, diss = draw_landscape(
            rng_land,
            p["N_PATHS"],
            p["BARRIER_MU"], p["BARRIER_SIGMA"],
            p["REORG_MU"], p["REORG_SIGMA"],
            p["DISS_MU"], p["DISS_SIGMA"],
            p["PROD_BARRIER_OFFSET"], p["PROD_DISS_OFFSET"],
        )

        for sj in range(SEEDS_PER_LANDSCAPE):
            seed_run = int(rng_master.integers(1, 2**31 - 1))
            out = run_once_chunked(p, barriers, reorg, diss, seed_run)

            row = [
                si, li, sj,
                p["N_PATHS"], p["BETA"], p["DISORDER_JUMP"], p["SHOCK_MODE"],
                p["GQR_FILTER_BASE"], p["GQR_FILTER_COST"], p["GQR_EARN_SCALE"],
                p["GQR_BUDGET0"], p["GQR_BUDGET_MIN"], p["GQR_BUDGET_MAX"], p["JURIS_THRESHOLD"],
                p["BARRIER_MU"], p["BARRIER_SIGMA"], p["REORG_MU"], p["REORG_SIGMA"], p["DISS_MU"], p["DISS_SIGMA"],
                p["PROD_BARRIER_OFFSET"], p["PROD_DISS_OFFSET"],
                p["WARSHEL_LOWER_PRODUCT"], p["WARSHEL_RAISE_WASTE"], p["WARSHEL_DISS_REDUCE"],
                out["y_arr"], out["y_mar"], out["y_war"], out["y_gqr"],
                out["dpp_arr"], out["dpp_mar"], out["dpp_war"], out["dpp_gqr"],
                out["gqr_budget"], out["gqr_final_filter"], out["gqr_dead"], out["final_disorder"],
            ]
            rows.append(row)

            run_count += 1
            now = time.time()
            if now - last_beat > 10:
                elapsed = now - t0
                print(f"progress: {run_count}/{total_runs} ({run_count/total_runs:5.1%}) elapsed={elapsed:6.1f}s")
                last_beat = now

rows = np.array(rows, dtype=float)

# Save everything needed for later plotting/analysis
np.savez(
    OUTFILE,
    rows=rows,
    columns=np.array(columns),

    # store the sampled parameter sets as a structured array for later grouping
    samples=np.array(samples, dtype=object),

    # store sweep ranges and meta
    ranges=np.array([RANGES], dtype=object),
    meta=np.array([{
        "EVENTS": EVENTS,
        "CHUNK": CHUNK,
        "N_CHUNKS": N_CHUNKS,
        "N_SAMPLES": N_SAMPLES,
        "LANDSCAPES_PER_SAMPLE": LANDSCAPES_PER_SAMPLE,
        "SEEDS_PER_LANDSCAPE": SEEDS_PER_LANDSCAPE,
        "SEED0": SEED0,
        "timestamp": time.time(),
    }], dtype=object),
)

print("Saved NPZ:", OUTFILE)
print("rows shape:", rows.shape)

progress: 406/9600 ( 4.2%) elapsed=  10.0s
progress: 943/9600 ( 9.8%) elapsed=  20.0s
progress: 1670/9600 (17.4%) elapsed=  30.0s
progress: 2268/9600 (23.6%) elapsed=  40.1s
progress: 2889/9600 (30.1%) elapsed=  50.1s
progress: 3616/9600 (37.7%) elapsed=  60.1s
progress: 4222/9600 (44.0%) elapsed=  70.1s
progress: 4959/9600 (51.7%) elapsed=  80.1s
progress: 5612/9600 (58.5%) elapsed=  90.1s
progress: 6189/9600 (64.5%) elapsed= 100.1s
progress: 6930/9600 (72.2%) elapsed= 110.1s
progress: 7573/9600 (78.9%) elapsed= 120.2s
progress: 8297/9600 (86.4%) elapsed= 130.2s
progress: 9058/9600 (94.4%) elapsed= 140.2s
progress: 9494/9600 (98.9%) elapsed= 150.2s
Saved NPZ: enz_governance_sweep_npz_only.npz
rows shape: (9600, 37)
